In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/5013 - Project/m5-forecasting-uncertainty

/content/drive/.shortcut-targets-by-id/129se4YWr7uxgjWW5UxAKQMxQvb6wH99Q/5013 - Project/m5-forecasting-uncertainty


In [3]:
import pandas as pd, numpy as np
from matplotlib import pyplot as plt
import scipy.stats  as stats

In [4]:
pd.options.display.max_columns = 50

In [5]:
best = pd.read_csv("../m5-forecasting-accuracy/Submission_LGBM.csv")
best.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.739274,0.686544,0.673668,0.707805,0.916229,0.847635,0.930423,0.851696,0.843188,0.865282,0.755220,0.979331,1.169826,1.052225,0.875713,0.837078,0.826894,0.841418,0.961785,1.261044,1.044882,0.781831,0.746933,0.761862,0.735168,0.954582,1.111383,1.067725
1,FOODS_1_001_CA_2_validation,0.924924,0.863886,0.875608,0.971936,0.911154,1.080291,1.393523,0.905049,0.913890,0.954387,0.920229,1.080633,1.331930,1.291469,0.959218,0.911372,0.973905,0.987461,1.134801,1.456729,1.459781,0.955668,0.944789,0.974691,1.009652,1.142520,1.454101,1.345215
2,FOODS_1_001_CA_3_validation,0.936745,0.827974,0.769283,0.749459,0.806185,1.048330,1.112415,1.033385,1.050957,1.009719,0.898557,1.120550,1.500319,1.374275,1.121280,0.989962,0.992332,0.973799,1.000253,1.346585,1.447919,1.014296,0.886542,0.874377,0.845866,0.985646,1.197012,1.213640
3,FOODS_1_001_CA_4_validation,0.419960,0.377348,0.380905,0.390079,0.425922,0.387764,0.439740,0.411590,0.424528,0.443911,0.368203,0.436571,0.469228,0.399305,0.383605,0.387331,0.395231,0.397146,0.471791,0.450463,0.429837,0.368815,0.350397,0.349890,0.356207,0.396154,0.415796,0.413200
4,FOODS_1_001_TX_1_validation,0.230222,0.237886,0.230396,0.236442,0.303618,0.196367,0.263038,0.846784,0.823671,0.761286,0.584143,0.918048,0.788217,0.487552,0.523639,0.523631,0.471141,0.470372,0.490310,0.629485,0.541189,0.434531,0.392101,0.376855,0.378761,0.437430,0.502436,0.475053


In [6]:
sales = pd.read_csv("./data/sales_train_validation.csv")
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,...,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [7]:
sub = best.merge(sales[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on = "id")
sub["_all_"] = "Total"
sub.shape

(30490, 35)

In [8]:
sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,item_id,dept_id,cat_id,store_id,state_id,_all_
0,FOODS_1_001_CA_1_validation,0.739274,0.686544,0.673668,0.707805,0.916229,0.847635,0.930423,0.851696,0.843188,0.865282,0.755220,0.979331,1.169826,1.052225,0.875713,0.837078,0.826894,0.841418,0.961785,1.261044,1.044882,0.781831,0.746933,0.761862,0.735168,0.954582,1.111383,1.067725,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,Total
1,FOODS_1_001_CA_2_validation,0.924924,0.863886,0.875608,0.971936,0.911154,1.080291,1.393523,0.905049,0.913890,0.954387,0.920229,1.080633,1.331930,1.291469,0.959218,0.911372,0.973905,0.987461,1.134801,1.456729,1.459781,0.955668,0.944789,0.974691,1.009652,1.142520,1.454101,1.345215,FOODS_1_001,FOODS_1,FOODS,CA_2,CA,Total
2,FOODS_1_001_CA_3_validation,0.936745,0.827974,0.769283,0.749459,0.806185,1.048330,1.112415,1.033385,1.050957,1.009719,0.898557,1.120550,1.500319,1.374275,1.121280,0.989962,0.992332,0.973799,1.000253,1.346585,1.447919,1.014296,0.886542,0.874377,0.845866,0.985646,1.197012,1.213640,FOODS_1_001,FOODS_1,FOODS,CA_3,CA,Total
3,FOODS_1_001_CA_4_validation,0.419960,0.377348,0.380905,0.390079,0.425922,0.387764,0.439740,0.411590,0.424528,0.443911,0.368203,0.436571,0.469228,0.399305,0.383605,0.387331,0.395231,0.397146,0.471791,0.450463,0.429837,0.368815,0.350397,0.349890,0.356207,0.396154,0.415796,0.413200,FOODS_1_001,FOODS_1,FOODS,CA_4,CA,Total
4,FOODS_1_001_TX_1_validation,0.230222,0.237886,0.230396,0.236442,0.303618,0.196367,0.263038,0.846784,0.823671,0.761286,0.584143,0.918048,0.788217,0.487552,0.523639,0.523631,0.471141,0.470372,0.490310,0.629485,0.541189,0.434531,0.392101,0.376855,0.378761,0.437430,0.502436,0.475053,FOODS_1_001,FOODS_1,FOODS,TX_1,TX,Total


In [9]:
qs = np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995])
qs.shape

def get_ratios(coef=0.15):
    qs2 = np.log(qs/(1-qs))*coef
    ratios = stats.norm.cdf(qs2)
    ratios /= ratios[4]
    ratios = pd.Series(ratios, index=qs)
    return ratios.round(3)


In [10]:
#coef between 0.05 and 0.24 is used, probably suboptimal values for now
level_coef_dict = {"id": get_ratios(coef=0.3), "item_id": get_ratios(coef=0.15),
                   "dept_id": get_ratios(coef=0.08), "cat_id": get_ratios(coef=0.07),
                   "store_id": get_ratios(coef=0.08), "state_id": get_ratios(coef=0.07), "_all_": get_ratios(coef=0.05),
                   ("state_id", "item_id"): get_ratios(coef=0.19),  ("state_id", "dept_id"): get_ratios(coef=0.1),
                    ("store_id","dept_id") : get_ratios(coef=0.11), ("state_id", "cat_id"): get_ratios(coef=0.08),
                    ("store_id","cat_id"): get_ratios(coef=0.1)
                  }

In [11]:
level_coef_dict["id"]

0.005    0.112
0.025    0.272
0.165    0.627
0.250    0.742
0.500    1.000
0.750    1.258
0.835    1.373
0.975    1.728
0.995    1.888
dtype: float64

In [12]:
level_coef_dict["cat_id"]

0.005    0.711
0.025    0.798
0.165    0.910
0.250    0.939
0.500    1.000
0.750    1.061
0.835    1.090
0.975    1.202
0.995    1.289
dtype: float64

In [13]:
def quantile_coefs(q, level):
    ratios = level_coef_dict[level]           
    return ratios.loc[q].values

In [14]:
def get_group_preds(pred, level):
    df = pred.groupby(level)[cols].sum()
    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    df[cols] *= quantile_coefs(q, level)[:, None]
    if level != "id":
        df["id"] = [f"{lev}_X_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    else:
        df["id"] = [f"{lev.replace('_validation', '')}_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    df = df[["id"]+list(cols)]
    return df

In [15]:
def get_couple_group_preds(pred, level1, level2):
    df = pred.groupby([level1, level2])[cols].sum()
    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    df[cols] *= quantile_coefs(q, (level1, level2))[:, None]
    df["id"] = [f"{lev1}_{lev2}_{q:.3f}_validation" for lev1,lev2, q in 
                zip(df[level1].values,df[level2].values, q)]
    df = df[["id"]+list(cols)]
    return df

In [ ]:
levels = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "_all_"]
couples = [("state_id", "item_id"),  ("state_id", "dept_id"),("store_id","dept_id"),
                            ("state_id", "cat_id"),("store_id","cat_id")]
cols = [f"F{i}" for i in range(1, 29)]

df = []
for level in levels :
    df.append(get_group_preds(sub, level))
for level1,level2 in couples:
    df.append(get_couple_group_preds(sub, level1, level2))
df = pd.concat(df, axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df,df] , axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df.loc[df.index >= len(df.index)//2, "id"] = df.loc[df.index >= len(df.index)//2, "id"].str.replace(
                                    "_validation", "_evaluation")

In [ ]:
print(df.shape)

df.head()

(771120, 29)


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_0.005_validation,0.082799,0.076893,0.075451,0.079274,0.102618,0.094935,0.104207,0.095390,0.094437,0.096912,0.084585,0.109685,0.131020,0.117849,0.098080,0.093753,0.092612,0.094239,0.107720,0.141237,0.117027,0.087565,0.083656,0.085329,0.082339,0.106913,0.124475,0.119585
1,FOODS_1_001_CA_2_0.005_validation,0.103591,0.096755,0.098068,0.108857,0.102049,0.120993,0.156075,0.101365,0.102356,0.106891,0.103066,0.121031,0.149176,0.144645,0.107432,0.102074,0.109077,0.110596,0.127098,0.163154,0.163496,0.107035,0.105816,0.109165,0.113081,0.127962,0.162859,0.150664
2,FOODS_1_001_CA_3_0.005_validation,0.104915,0.092733,0.086160,0.083939,0.090293,0.117413,0.124590,0.115739,0.117707,0.113088,0.100638,0.125502,0.168036,0.153919,0.125583,0.110876,0.111141,0.109065,0.112028,0.150817,0.162167,0.113601,0.099293,0.097930,0.094737,0.110392,0.134065,0.135928
3,FOODS_1_001_CA_4_0.005_validation,0.047036,0.042263,0.042661,0.043689,0.047703,0.043430,0.049251,0.046098,0.047547,0.049718,0.041239,0.048896,0.052554,0.044722,0.042964,0.043381,0.044266,0.044480,0.052841,0.050452,0.048142,0.041307,0.039244,0.039188,0.039895,0.044369,0.046569,0.046278
4,FOODS_1_001_TX_1_0.005_validation,0.025785,0.026643,0.025804,0.026482,0.034005,0.021993,0.029460,0.094840,0.092251,0.085264,0.065424,0.102821,0.088280,0.054606,0.058648,0.058647,0.052768,0.052682,0.054915,0.070502,0.060613,0.048667,0.043915,0.042208,0.042421,0.048992,0.056273,0.053206


In [ ]:
df.to_csv("from_accuracy2uncertainty_LGBM.csv", index = False)